In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import phik
from IPython.display import Markdown as md

In [5]:
# Insert the dataset as data frame
df = pd.read_pickle(r"shared/Motor_Vehicle_Collisions_-_Crashes.pkl")
df.head()

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN


In [6]:
# Which borough has had the second highest total number of crashes reported since 2012? (Ignore missing values)
bor = df['BOROUGH'].value_counts()
#time = df['CRASH DATE_CRASH TIME'].value_counts()
print(bor)

BOROUGH
BROOKLYN         441026
QUEENS           372457
MANHATTAN        313266
BRONX            205345
STATEN ISLAND     58297
Name: count, dtype: int64


In [2]:
# Considering the entire data set since July 2012, which borough has the most crashes for every 100,000 people? (hint: divide the total count of crashes by the population, and multiply by 100,000)
populations = {
    'BRONX': 1446788,
    'BROOKLYN': 2648452,
    'MANHATTAN': 1638281,
    'QUEENS': 2330295,
    'STATEN ISLAND': 487155
}

crashes = {
    'BROOKLYN': 441026,
    'QUEENS': 372457,
    'MANHATTAN': 313266,
    'BRONX': 205345,
    'STATEN ISLAND': 58297
}

crashes_per_100k = {borough: (crashes[borough] / populations[borough]) * 100000 for borough in populations}
most_crashes = max(crashes_per_100k, key=crashes_per_100k.get)
most_crashes, crashes_per_100k[most_crashes]

('MANHATTAN', 19121.628096767283)

In [7]:
# What is the leading cause of crashes (ignoring 'Unspecified') (use the field 'CONTRIBUTING FACTOR VEHICLE 1')?
# the top 5 causes of crashes (ignoring 'Unspecified') account for what proportion of total crashes?
df_filtered = df[df['CONTRIBUTING FACTOR VEHICLE 1'] != 'Unspecified']

leading_cause = df_filtered['CONTRIBUTING FACTOR VEHICLE 1'].value_counts().idxmax()
top_5_causes = df_filtered['CONTRIBUTING FACTOR VEHICLE 1'].value_counts().head(5)
total_crashes = len(df_filtered)
proportion_top_5 = top_5_causes.sum() / total_crashes

print(f"The leading cause of crashes is: {leading_cause}")
print(f"The top 5 causes account for {proportion_top_5:.2%} of total crashes.")

The leading cause of crashes is: Driver Inattention/Distraction
The top 5 causes account for 57.76% of total crashes.


In [8]:
# To see the full list of all the categories
print(df.columns.tolist())

['CRASH DATE_CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME', 'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']


In [9]:
# Considering the entire data since 2012, what is the total count of accidents that involved two or more fatalities?  (Missing values for the number of persons killed should be assumed to be equal to zero killed.)
df['NUMBER OF PERSONS KILLED'] = df['NUMBER OF PERSONS KILLED'].fillna(0)
fatal_accidents2 = df[df['NUMBER OF PERSONS KILLED'] >= 2]
total_fatal_accidents = len(fatal_accidents2)

print(f"The total count of accidents that involved two or more fatalities is: {total_fatal_accidents}")

The total count of accidents that involved two or more fatalities is: 88


In [10]:
# On average, for every 1000 accidents, how many have resulted in at least one person dead?
fatal_accidents1 = df[df['NUMBER OF PERSONS KILLED'] >= 1]
fatal_accidents_proportion = (len(fatal_accidents1) / len(df)) * 1000

print(f"On average, for every 1000 accidents, {fatal_accidents_proportion:.2f} have resulted in at least one person dead.")

On average, for every 1000 accidents, 1.39 have resulted in at least one person dead.


In [11]:
# What proportion of accidents in the data do not have a Cross Street Name?
missing_cross_street_count = df['CROSS STREET NAME'].isnull().sum()
total_accidents = len(df)
proportion_missing_cross_street = (missing_cross_street_count / total_accidents) * 100

print(f"The proportion of accidents without a 'CROSS STREET NAME' is {proportion_missing_cross_street:.2f}%.")

The proportion of accidents without a 'CROSS STREET NAME' is 37.44%.


In [13]:
# The fields 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2' represent the first two vehicles involved in the accident. Which combination of vehicles have the most number of accidents?
vehicle_combinations = df.groupby(['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2']).size()
most_common_combination = vehicle_combinations.idxmax()

print(f"The most common combination of vehicles involved in accidents is: {most_common_combination}")

The most common combination of vehicles involved in accidents is: ('Sedan', 'Sedan')


In [14]:
# Among crashes where the contributing factor (CONTRIBUTING FACTOR VEHICLE 1) was alcohol involvement, what proportion resulted in a fatality?
alcohol_related = df[df['CONTRIBUTING FACTOR VEHICLE 1'] == 'Alcohol Involvement']
fatal_accidents = alcohol_related[alcohol_related['NUMBER OF PERSONS KILLED'] > 0]
proportion_fatal = len(fatal_accidents) / len(alcohol_related)

print(f"Proportion of alcohol-related crashes that resulted in a fatality: {proportion_fatal:.4f}")

Proportion of alcohol-related crashes that resulted in a fatality: 0.0047


In [15]:
# What proportion of crashes occur during the evening rush hour, defined as starting at 4 PM, and before 7 PM?
Fourseven = df[(df['CRASH DATE_CRASH TIME'].dt.hour >= 16) & (df['CRASH DATE_CRASH TIME'].dt.hour < 19)]
proportion_crash = len(Fourseven) / len(df)

print(f"The proportion of crashes occur during the evening rush hour: {proportion_crash}")

The proportion of crashes occur during the evening rush hour: 0.20514010935243243


In [36]:
# Among crashes involving motorcycles, what proportion resulted in injuries but no fatalities?
motorcycle_crashes = df[(df['VEHICLE TYPE CODE 1'].str.contains('MOTORCYCLE', case=False, na=False)) |
                        (df['VEHICLE TYPE CODE 2'].str.contains('MOTORCYCLE', case=False, na=False))]

# Make sure 'NUMBER OF PERSONS INJURED' and 'NUMBER OF PERSONS KILLED' are treated as integers, fill missing values with 0
motorcycle_crashes.loc[:, 'NUMBER OF PERSONS INJURED'] = motorcycle_crashes['NUMBER OF PERSONS INJURED'].fillna(0)
motorcycle_crashes.loc[:, 'NUMBER OF PERSONS KILLED'] = motorcycle_crashes['NUMBER OF PERSONS KILLED'].fillna(0)

injuries_no_fatalities = motorcycle_crashes[
    (motorcycle_crashes['NUMBER OF PERSONS INJURED'] > 0) &
    (motorcycle_crashes['NUMBER OF PERSONS KILLED'] == 0)]
proportion = (len(injuries_no_fatalities) / len(motorcycle_crashes))

print(f"Proportion of motorcycle crashes that resulted in injuries but no fatalities: {proportion:.4f}")

Proportion of motorcycle crashes that resulted in injuries but no fatalities: 55.6879


In [39]:
motorcycle_crashes = df[(df['VEHICLE TYPE CODE 1'].str.contains('MOTORCYCLE', na=False)) |
                        (df['VEHICLE TYPE CODE 2'].str.contains('MOTORCYCLE', na=False))]

motorcycle_crashes.loc[:, 'NUMBER OF PERSONS INJURED'] = motorcycle_crashes['NUMBER OF PERSONS INJURED'].fillna(0)
motorcycle_crashes.loc[:, 'NUMBER OF PERSONS KILLED'] = motorcycle_crashes['NUMBER OF PERSONS KILLED'].fillna(0)

injuries_no_fatalities = motorcycle_crashes[(motorcycle_crashes['NUMBER OF PERSONS INJURED'] > 0) & 
                                            (motorcycle_crashes['NUMBER OF PERSONS KILLED'] == 0)].shape[0]

total_motorcycle_crashes = motorcycle_crashes.shape[0]

proportion_injuries_no_fatalities = (injuries_no_fatalities / total_motorcycle_crashes) * 100
proportion_injuries_no_fatalities

50.04565018912221

In [17]:
# How many crashes involved bicycles as one of the vehicles?
bicycle_crashes = df[(df['VEHICLE TYPE CODE 1'].str.contains('BICYCLE', case=False, na=False)) |
                     (df['VEHICLE TYPE CODE 2'].str.contains('BICYCLE', case=False, na=False))]
num_bicycle_crashes = len(bicycle_crashes)

print(f"Number of crashes involving bicycles: {num_bicycle_crashes}")

Number of crashes involving bicycles: 19108
